Sau khi dùng Spotify API và xử lý dữ liệu, nhóm đã thu thập được 688431 dòng dữ liệu. Tuy nhiên nhóm vẫn muốn thu thập thêm một vài trường dữ liệu hữu ích cho việc phân tích dữ liệu. Do đó nhóm sử dụng thêm 2 API là Genius API và Rapid API để thu thập một số trường dữ liệu hữu ích như lyrics, ...

In [ ]:
pip install fake-useragent

### Chọn ra 10000 bài hát tiêu biểu

Vì data gốc thu thập được là 689400 dòng, tuy nhiên Rapid API chỉ giới hạn mỗi lần dùng là 100 bài hát. Do đó nhóm quyết định chọn ra 10.000 bài ngẫu nhiên nhưng phân bố đều ở 10 khoảng điểm popularity (từ 0 tới 100) để phân tích dữ liệu

In [ ]:
import pandas as pd
import numpy as np
import time
import json
import os
import sys
import random
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from fake_useragent import UserAgent
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [ ]:
eda_df = pd.read_csv('/kaggle/input/eda-data-test/eda_data_withoutgeniusID.csv')

In [ ]:
eda_df[:10]

In [ ]:
eda_df_first = eda_df[:5000]
eda_df_first

In [ ]:
# Setup random User-Agent
user_agents = UserAgent()

In [ ]:
def get_genre_with_header(song, artist):
    url = "https://itunes.apple.com/search"
    params = {"term": f"{song} {artist}", "entity": "song", "limit": 1}
    headers = {"User-Agent": user_agents.random}  # Randomized User-Agent
    
    session = requests.Session()
    retries = Retry(total=5, backoff_factor=2, status_forcelist=[429, 500, 502, 503, 504])
    session.mount('https://', HTTPAdapter(max_retries=retries))
    
    try:
        response = session.get(url, params=params, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data.get("resultCount", 0) > 0:
                return data["results"][0].get("primaryGenreName", "")
            return "Not Found"
        else:
            print(f"API error {response.status_code}: {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    return "Error"

In [ ]:
# Fetch genres with sleep and batch delay
genres = []
for i, (song, artist) in enumerate(zip(eda_df_first['track_name'], eda_df_first['artist_name'])):
    genre = get_genre_with_header(song, artist)
    genres.append(genre)
    print(i, genre)

    time.sleep(random.uniform(0.8, 1.5))  # Random delay to prevent detection

    if i > 0 and i % 30 == 0:
        print("Taking a longer break...")
        time.sleep(random.uniform(15, 25))  # Periodic longer delay

In [ ]:
# Assign 'genre' column back to DataFrame
eda_df_first['genre'] = genres

In [ ]:
eda_df_first.head()

In [ ]:
eda_df_first.to_csv('/kaggle/working/eda_df_first_genre.csv', index=False)

In [ ]:
import musicbrainzngs

# Thiết lập thông tin ứng dụng của bạn
musicbrainzngs.set_useragent("YourAppName", "1.0", "your-email@example.com")

def get_genre_from_musicbrainz(song_name, artist_name):
    result = musicbrainzngs.search_recordings(recording=song_name, artist=artist_name, limit=1)
    if result['recording-list']:
        recording = result['recording-list'][0]
        if 'tag-list' in recording:
            return [tag['name'] for tag in recording['tag-list']]
    print("Không tìm thấy bài hát trên MusicBrainz.")
    return None

# Ví dụ sử dụng
genre = get_genre_from_musicbrainz('Shape of You', 'Ed Sheeran')
print(genre)
